# Setup

In [1]:
%pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.6 MB/s eta 0:00:00
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39588 sha256=ff2652c3c66b604011a5acf69b8cd1b9eb528e3b362ca927e329808f7e8a5b14
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [2]:
import torch

device = torch.device("cpu")

In [3]:
from spuco.utils import set_seed

set_seed(0)

# Dataset Creation

In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.0MB/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 497kB/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.88MB/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.89MB/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



In [5]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


In [6]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [7]:
from spuco.models import model_factory
from spuco.robust_train import ERM
from torch.optim import SGD
import numpy as np
from spuco.robust_train import CustomSampleERM
from spuco.utils import Trainer
from spuco.group_inference import JTTInference

# Train using JTT

In [8]:
#jtt params
jtt_epochs = 2
print('Total trainset size: ' + str(len(trainset)))
print('Mini trainset size: ' + str(len(trainset)/4))

Total trainset size: 48004
Mini trainset size: 12001.0


## Group Inference Merge: 3 Groups

In [9]:
def gi_subset(num_partitions, partition_id, trainset, device):
  #generate subset index
  partition_len = int(len(trainset)/num_partitions)
  train_index = np.arange(partition_len*partition_id, partition_len*partition_id+partition_len)
  train_index = train_index.tolist()
  print(train_index)
  #train on subset
  infer_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
  gi_trainer = CustomSampleERM(
      model=infer_model,
      trainset=trainset,
      batch_size=64,
      optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
      num_epochs=2,
      indices=train_index,
      device=device,
      verbose=True
  )
  gi_trainer.train()
  #get labels
  gi_trainer_results = Trainer(
    model=infer_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
  )
  output = gi_trainer_results.get_trainset_outputs()
  max_output = torch.max(output, dim=1)
  pred_labels = max_output.indices
  pred_labels = pred_labels.tolist()
  actual_labels = trainset.labels
  #jtt inference on subset
  start_ind = partition_len*partition_id
  end_ind = partition_len*partition_id+partition_len
  jtt = JTTInference(pred_labels[start_ind:end_ind], actual_labels[start_ind:end_ind])
  gi = jtt.infer_groups()
  nominal_upsample = len(gi[(0, 0)])/len(gi[(0, 1)])
  print('\nNominal Upsample: ' + str(nominal_upsample))
  print('Majority class size: ' + str(len(gi[(0, 0)])))
  print('Minority class size: ' + str(len(gi[(0, 1)])))
  #return majority, minority group indices
  majority = [i + partition_len*partition_id for i in gi[(0, 0)]]
  minority = [i + partition_len*partition_id for i in gi[(0, 1)]]
  return majority, minority

In [10]:
maj1, min1 = gi_subset(3, 0, trainset, device)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

Epoch 0:   3%|▎         | 8/251 [00:00<00:18, 13.10batch/s, accuracy=26.5625%, loss=1.59]

 | Epoch 0 | Loss: 1.6096055507659912 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6306883096694946 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6058577299118042 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.6084169149398804 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6087136268615723 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6017506122589111 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6221816539764404 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6143782138824463 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.5987790822982788 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.611764669418335 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6116615533828735 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.605337142944336 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.5910284519195557 | Accuracy: 26.5625%


Epoch 0:   9%|▉         | 22/251 [00:01<00:07, 32.55batch/s, accuracy=40.625%, loss=1.59] 

...

 | Epoch 1 | Loss: 0.14767315983772278 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.009059937670826912 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.010845947079360485 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.009786411188542843 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.11093324422836304 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.008973457850515842 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007104734890162945 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006967230699956417 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007020811550319195 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.10608071088790894 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.006324253976345062 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004901334643363953 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004761311691254377 | Accuracy: 100.0%


Epoch 1: 100%|██████████| 251/251 [00:05<00:00, 49.00batch/s, accuracy=100.0%, loss=0.00957]


 | Epoch 1 | Loss: 0.1136845201253891 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.005512382835149765 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.0045957062393426895 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.009570550173521042 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:05<00:00, 138.49batch/s]


Nominal Upsample: 172.92391304347825
Majority class size: 15909
Minority class size: 92


In [11]:
maj2, min2 = gi_subset(3, 1, trainset, device)

[16001, 16002, 16003, 16004, 16005, 16006, 16007, 16008, 16009, 16010, 16011, 16012, 16013, 16014, 16015, 16016, 16017, 16018, 16019, 16020, 16021, 16022, 16023, 16024, 16025, 16026, 16027, 16028, 16029, 16030, 16031, 16032, 16033, 16034, 16035, 16036, 16037, 16038, 16039, 16040, 16041, 16042, 16043, 16044, 16045, 16046, 16047, 16048, 16049, 16050, 16051, 16052, 16053, 16054, 16055, 16056, 16057, 16058, 16059, 16060, 16061, 16062, 16063, 16064, 16065, 16066, 16067, 16068, 16069, 16070, 16071, 16072, 16073, 16074, 16075, 16076, 16077, 16078, 16079, 16080, 16081, 16082, 16083, 16084, 16085, 16086, 16087, 16088, 16089, 16090, 16091, 16092, 16093, 16094, 16095, 16096, 16097, 16098, 16099, 16100, 16101, 16102, 16103, 16104, 16105, 16106, 16107, 16108, 16109, 16110, 16111, 16112, 16113, 16114, 16115, 16116, 16117, 16118, 16119, 16120, 16121, 16122, 16123, 16124, 16125, 16126, 16127, 16128, 16129, 16130, 16131, 16132, 16133, 16134, 16135, 16136, 16137, 16138, 16139, 16140, 16141, 16142, 16143

Epoch 0:   2%|▏         | 6/251 [00:00<00:13, 18.64batch/s, accuracy=15.625%, loss=1.61]

 | Epoch 0 | Loss: 1.6173495054244995 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6272523403167725 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6100497245788574 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6201503276824951 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.625586748123169 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6053707599639893 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.612199068069458 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.612296223640442 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6079273223876953 | Accuracy: 15.625%


Epoch 0:   6%|▌         | 14/251 [00:00<00:08, 29.18batch/s, accuracy=56.25%, loss=1.59]

 | Epoch 0 | Loss: 1.6107556819915771 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6024401187896729 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.5961071252822876 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6092102527618408 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.5958000421524048 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.5979560613632202 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.5987061262130737 | Accuracy: 37.5%
 | Epoch 0 | Loss: 1.5927965641021729 | Accuracy: 56.25%


...

 | Epoch 1 | Loss: 0.01105959340929985 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.13992783427238464 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.010468480177223682 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.12694092094898224 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.08939061313867569 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.008163952268660069 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.00763983279466629 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007865093648433685 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.0067396098747849464 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006494690198451281 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.11280190199613571 | Accuracy: 98.4375%


Epoch 1: 100%|██████████| 251/251 [00:05<00:00, 46.19batch/s, accuracy=100.0%, loss=0.00692]


 | Epoch 1 | Loss: 0.0993179902434349 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.005524524487555027 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.005917475558817387 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.005823427811264992 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004606533795595169 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006922074127942324 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:05<00:00, 127.21batch/s]


Nominal Upsample: 155.87254901960785
Majority class size: 15899
Minority class size: 102


In [12]:
maj3, min3 = gi_subset(3, 2, trainset, device)

[32002, 32003, 32004, 32005, 32006, 32007, 32008, 32009, 32010, 32011, 32012, 32013, 32014, 32015, 32016, 32017, 32018, 32019, 32020, 32021, 32022, 32023, 32024, 32025, 32026, 32027, 32028, 32029, 32030, 32031, 32032, 32033, 32034, 32035, 32036, 32037, 32038, 32039, 32040, 32041, 32042, 32043, 32044, 32045, 32046, 32047, 32048, 32049, 32050, 32051, 32052, 32053, 32054, 32055, 32056, 32057, 32058, 32059, 32060, 32061, 32062, 32063, 32064, 32065, 32066, 32067, 32068, 32069, 32070, 32071, 32072, 32073, 32074, 32075, 32076, 32077, 32078, 32079, 32080, 32081, 32082, 32083, 32084, 32085, 32086, 32087, 32088, 32089, 32090, 32091, 32092, 32093, 32094, 32095, 32096, 32097, 32098, 32099, 32100, 32101, 32102, 32103, 32104, 32105, 32106, 32107, 32108, 32109, 32110, 32111, 32112, 32113, 32114, 32115, 32116, 32117, 32118, 32119, 32120, 32121, 32122, 32123, 32124, 32125, 32126, 32127, 32128, 32129, 32130, 32131, 32132, 32133, 32134, 32135, 32136, 32137, 32138, 32139, 32140, 32141, 32142, 32143, 32144

Epoch 0:   2%|▏         | 6/251 [00:00<00:13, 17.64batch/s, accuracy=25.0%, loss=1.6]    

 | Epoch 0 | Loss: 1.6182113885879517 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.611406922340393 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6173381805419922 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6267026662826538 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6151278018951416 | Accuracy: 35.9375%
 | Epoch 0 | Loss: 1.6117277145385742 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6095714569091797 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.609755039215088 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.598389744758606 | Accuracy: 25.0%

Epoch 0:   6%|▌         | 15/251 [00:00<00:08, 29.41batch/s, accuracy=15.625%, loss=1.6] 


 | Epoch 0 | Loss: 1.6188963651657104 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.5953038930892944 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.5996240377426147 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.5942217111587524 | Accuracy: 32.8125%
 | Epoch 0 | Loss: 1.60610830783844 | Accuracy: 34.375%
 | Epoch 0 | Loss: 1.5996962785720825 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6002393960952759 | Accuracy: 15.625%


...

 | Epoch 1 | Loss: 0.010412812232971191 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.011496645398437977 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.14200983941555023 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.009316207841038704 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.35394710302352905 | Accuracy: 96.875%
 | Epoch 1 | Loss: 0.010175269097089767 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007840176112949848 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.008738844655454159 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.12566804885864258 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.008274714462459087 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007640897296369076 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.28386256098747253 | Accuracy: 95.3125%
 | Epoch 1 | Loss: 0.12015131115913391 | Accuracy: 98.4375%


Epoch 1: 100%|██████████| 251/251 [00:04<00:00, 51.97batch/s, accuracy=100.0%, loss=0.0122]


 | Epoch 1 | Loss: 0.012399581260979176 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.012472529895603657 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.01268041506409645 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.010744080878794193 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.012044783681631088 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.11044607311487198 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.012166455388069153 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:05<00:00, 126.75batch/s]


Nominal Upsample: 163.95876288659792
Majority class size: 15904
Minority class size: 97


In [13]:
min = min1.copy()
min.extend(min2)
min.extend(min3)
maj = maj1.copy()
maj.extend(maj2)
maj.extend(maj3)
upsampled_indices = [i for i in min for x in range(100)]
upsampled_indices.extend(maj)
len(upsampled_indices)

76812

## Robust Training

In [14]:
from spuco.robust_train import CustomSampleERM

robust_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
rt_erm = CustomSampleERM(
    model=robust_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(robust_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    num_epochs=3,
    indices=upsampled_indices,
    device=device,
    verbose=True
)
rt_erm.train()

Epoch 0:   0%|          | 6/1201 [00:00<01:20, 14.83batch/s, accuracy=15.625%, loss=1.6]

 | Epoch 0 | Loss: 1.6250419616699219 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6137940883636475 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6132230758666992 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6238666772842407 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.620457649230957 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.608651041984558 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6092255115509033 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.604673981666565 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6006380319595337 | Accuracy: 15.625%


Epoch 0:   1%|▏         | 16/1201 [00:00<00:39, 29.76batch/s, accuracy=15.625%, loss=1.61]

 | Epoch 0 | Loss: 1.608812689781189 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.601920247077942 | Accuracy: 39.0625%
 | Epoch 0 | Loss: 1.6191245317459106 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6087976694107056 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6077842712402344 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.6107919216156006 | Accuracy: 7.8125%
 | Epoch 0 | Loss: 1.6074031591415405 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.603069543838501 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.608484148979187 | Accuracy: 15.625%


Epoch 0:   2%|▏         | 26/1201 [00:00<00:31, 37.79batch/s, accuracy=23.4375%, loss=1.61]

...

Epoch 2:  99%|█████████▉| 1192/1201 [00:28<00:00, 35.84batch/s, accuracy=96.875%, loss=0.0986] 

 | Epoch 2 | Loss: 0.004087699111551046 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0017699378076940775 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.012743141502141953 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0017901184037327766 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0015720921801403165 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.004962865728884935 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0005570109933614731 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.09862057119607925 | Accuracy: 96.875%


Epoch 2: 100%|█████████▉| 1197/1201 [00:28<00:00, 39.29batch/s, accuracy=100.0%, loss=0.000168]

 | Epoch 2 | Loss: 0.00218227063305676 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.001675796345807612 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0005334471352398396 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.012607286684215069 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0017257550498470664 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.041013203561306 | Accuracy: 98.4375%
 | Epoch 2 | Loss: 0.0001682494767010212 | Accuracy: 100.0%


Epoch 2: 100%|██████████| 1201/1201 [00:28<00:00, 42.02batch/s, accuracy=100.0%, loss=0.000168]


In [15]:
from spuco.evaluate import Evaluator

rt_evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=robust_model,
    device=device,
    verbose=True
)
rt_evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:13,  1.76it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:10,  2.26it/s]

Group (0, 1) Accuracy: 58.628841607565015


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:08,  2.67it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:07<00:00,  3.54it/s]

Group (4, 3) Accuracy: 62.121212121212125


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:07<00:00,  3.33it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 58.628841607565015,
 (0, 2): 75.177304964539,
 (0, 3): 71.39479905437352,
 (0, 4): 69.03073286052009,
 (1, 0): 66.01466992665037,
 (1, 1): 99.75550122249389,
 (1, 2): 81.12745098039215,
 (1, 3): 69.11764705882354,
 (1, 4): 55.14705882352941,
 (2, 0): 74.66666666666667,
 (2, 1): 47.46666666666667,
 (2, 2): 100.0,
 (2, 3): 68.26666666666667,
 (2, 4): 32.62032085561497,
 (3, 0): 55.27638190954774,
 (3, 1): 43.8287153652393,
 (3, 2): 74.3073047858942,
 (3, 3): 99.74811083123426,
 (3, 4): 63.22418136020151,
 (4, 0): 67.75818639798489,
 (4, 1): 18.639798488664987,
 (4, 2): 35.012594458438286,
 (4, 3): 62.121212121212125,
 (4, 4): 100.0}

In [16]:
rt_evaluator.worst_group_accuracy

((4, 1), 18.639798488664987)

In [17]:
rt_evaluator.average_accuracy

99.66232641155617

In [18]:
rt_evaluator.evaluate_spurious_attribute_prediction()

19.94